In [59]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import  FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [60]:
vectorstore = FAISS.from_texts(
    ["harison worked at kensho"], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)
model = ChatOpenAI()

In [61]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [62]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [63]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [64]:
chain.invoke({"question": "where did harrsion work", "language": "korean"})

'Question: 하리슨은 어디에서 일했나요? \n\nAnswer: 하리슨은 켄쇼에서 일했습니다.'

## Converstaional Retrieval Chain

In [65]:
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel

In [66]:
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and  a follow up question, rephrase the follow up question to be standalone question, in its orginal language.

Chat Hisotry:
{chat_history}
Follow up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [67]:
template = """Answer the qeustion based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [68]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [ format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [69]:
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)

_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"]
}

conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

In [70]:
conversational_qa_chain.invoke(
    {
        "question": "where did harrison work?",
        "chat_history": [],
    }
)

AIMessage(content='Harrison worked at Kensho.')

In [71]:
conversational_qa_chain.invoke(
    {
        "question": "where did he work?",
        "chat_history": [
            HumanMessage(content="Who worte this notebook"),
            AIMessage(content="Harrison")
        ],
    }
)

AIMessage(content='Harrison worked at Kensho.')

### With Memory and returning source documents

In [72]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

In [73]:
memory = ConversationBufferMemory(
    return_message=True, output_key="answer", input_key="question"
)

In [74]:
# First we add a step to load memory
# This add a 'memory' key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
)
# Now we calculate teh standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"])
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
}

# Now we retrieve the docuemnts
retrieved_docuemnts = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}

# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question")
}

# Add finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(),
    "docs": itemgetter("docs")
}

# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_docuemnts | answer




In [75]:
inputs = {"question": "where did harrison work?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Harrison worked at Kensho.'),
 'docs': [Document(page_content='harison worked at kensho')]}

In [76]:
# Note that the memory does not save automatically
# This will be improved in the future.
# For now, you need to save it yourself
memory.save_context(inputs, {"answer": result["answer"].content})

In [77]:
memory.load_memory_variables({})

{'history': 'Human: where did harrison work?\nAI: Harrison worked at Kensho.'}

In [78]:
inputs = {"question": "but where did he really work?"}
result = final_chain.invoke(inputs)
result

ValueError: Got unsupported message type: H